In [41]:
import pandas as pd
import numpy as np

iso_data = pd.read_json('../js_files/iso6393.json')
wiki_data = pd.read_csv('../csv_files/wiki_languages_most_recent.csv', index_col = 0)
extinct_data = pd.read_csv('../csv_files/extinct_languages_with_info.csv', index_col = 0)
unesco_data = pd.read_csv('../csv_files/Extinct languages - DATA SUMMARY.csv')
lat_long_dialects = pd.read_csv('../csv_files/languages_and_dialects_geo.csv', index_col = 0)
lang_regions = pd.read_csv('../csv_files/languages_by_region.csv')

In [42]:
for i in iso_data.value_counts('iso6393').to_list():
    if i > 1:
        print(i)
        print(iso_data[iso_data['iso6393'] == i])

In [43]:
wiki_altered = wiki_data.drop_duplicates(subset = ['goglottocodettolog'])
wiki_new = wiki_altered + wiki_data[wiki_data['goglottocodettolog'].isna()]

In [44]:
wiki_new = wiki_new.rename(columns = {'lang': 'name'})
# print(wiki_new.columns)
# print(iso_data.columns)
merge_v1 = pd.merge(iso_data, wiki_new, left_on = 'name', right_on = 'name', how = 'left')
merge_v1 = merge_v1.drop(columns = 'iso3code')
# merge_v1 

In [45]:
extinct_data = extinct_data.rename(columns = {'lang': 'name'})
merge_v1 = merge_v1.rename(columns = {'goglottocodettolog': 'glottocode'})
merge_v2 = pd.merge(merge_v1, extinct_data, left_on = 'name', right_on = 'name', how = 'left')
merge_v2 = merge_v2.drop_duplicates(subset = ['name'])
merge_v2 = merge_v2.drop(columns = 'iso3code')

merge_v2['family'] = merge_v2.apply(lambda x: x['family_x'] if x['family_x'] is not np.NaN else x['family_y'], axis = 1)
merge_v2['dialects'] = merge_v2.apply(lambda x: x['dialects_x'] if x['dialects_x'] is not np.NaN else x['dialects_y'], axis = 1)
merge_v2['glottocode'] = merge_v2.apply(lambda x: x['glottocode_x'] if x['glottocode_x'] is not np.NaN else x['glottocode_y'], axis = 1)
merge_v2['speakers'] = merge_v2.apply(lambda x: x['speakers_x'] if x['speakers_x'] is not np.NaN else x['speakers_y'], axis = 1)
merge_v2['regions'] = merge_v2.apply(lambda x: x['regions_x'] if x['regions_x'] is not np.NaN else x['regions_y'], axis = 1)
merge_v2['off_lang'] = merge_v2.apply(lambda x: x['off_lang_x'] if x['off_lang_x'] is not np.NaN else x['off_lang_y'], axis = 1)
merge_v2['rec_min_lang'] = merge_v2.apply(lambda x: x['rec_min_lang_x'] if x['rec_min_lang_x'] is not np.NaN else x['rec_min_lang_y'], axis = 1)
merge_v2['url'] = merge_v2.apply(lambda x: x['url_x'] if x['url_x'] is not np.NaN else x['url_y'], axis = 1)

merge_v2 = merge_v2.drop(columns = ['family_x', 'family_y', 'dialects_x', 'dialects_y', 'glottocode_x', 'glottocode_y', 'speakers_x', 'speakers_y', 'regions_x', 'regions_y', 'off_lang_x', 'off_lang_y', 'rec_min_lang_x', 'rec_min_lang_y', 'url_x', 'url_y'])

# merge_v2

In [46]:
unesco_data = unesco_data.rename(columns = {'Number of speakers': 'speakers'})
unesco_data = unesco_data.rename(columns = {'Name in English': 'name'})

merge_v3 = pd.merge(merge_v2, unesco_data, left_on = 'name', right_on = 'name', how = 'left')
merge_v3['speakers'] = merge_v3.apply(lambda x: x['speakers_y'] if x['speakers_y'] is not np.NaN else x['speakers_x'], axis = 1)
merge_v3 = merge_v3.drop(columns = ['speakers_x', 'speakers_y'])
merge_v3

,name,type,scope,iso6393,family,dialects,glottocode,regions,off_lang,rec_min_lang,url,Degree of endangerment,speakers
0,Ghotuo,living,individual,aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alumu-Tesu,living,individual,aab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ari,living,individual,aac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Severely endangered,80.0
3,Amal,living,individual,aad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arbëreshë Albanian,living,individual,aae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7864,Youjiang Zhuang,living,individual,zyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7865,Yongnan Zhuang,living,individual,zyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7866,Zyphe Chin,living,individual,zyp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7867,Zaza,living,macrolanguage,zza,"Indo-European, Indo-Iranian, Iranian, Western,...","Indo-European, Indo-Iranian, Iranian, Western,...",zaza1246,"Provinces of Sivas, Tunceli, Bingöl, Erzurum, ...",Unknown,Unknown,https://en.wikipedia.org/wiki/Zaza_language,NaN,NaN


In [47]:
lat_long_dialects = lat_long_dialects.rename(columns = {'isocodes': 'iso6393'})
lat_long_dialects = lat_long_dialects.reset_index()

In [48]:
# print(lat_long_dialects.columns)
# print(merge_v3.columns)

merge_v4 = pd.merge(merge_v3, lat_long_dialects, left_on = 'iso6393', right_on = 'iso6393', how = 'left')
merge_v4['glottocode'] = merge_v4.apply(lambda x: x['glottocode_y'] if x['glottocode_y'] is not np.NaN else x['glottocode_x'], axis = 1)
merge_v4 = merge_v4.drop(columns = ['glottocode_x', 'glottocode_y'])
# merge_v4 = merge_v4.drop(columns = ['isocodes'])
merge_v4

,name_x,type,scope,iso6393,family,dialects,regions,off_lang,rec_min_lang,url,Degree of endangerment,speakers,name_y,level,macroarea,latitude,longitude,glottocode
0,Ghotuo,living,individual,aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ghotuo,language,Africa,7.11551,5.95663,ghot1243
1,Alumu-Tesu,living,individual,aab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arum,language,Africa,9.01513,8.55956,alum1246
2,Ari,living,individual,aac,NaN,NaN,NaN,NaN,NaN,NaN,Severely endangered,80.0,Ari,language,Papunesia,-7.95526,142.40000,arii1243
3,Amal,living,individual,aad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Amal,language,Papunesia,-4.04517,141.99500,amal1242
4,Arbëreshë Albanian,living,individual,aae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arbëreshë Albanian,language,Eurasia,38.91100,16.71640,arbe1236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7864,Youjiang Zhuang,living,individual,zyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Youjiang Zhuang,language,Eurasia,23.75000,106.90000,youj1238
7865,Yongnan Zhuang,living,individual,zyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yongnan Zhuang,language,Eurasia,22.80000,108.00000,yong1275
7866,Zyphe Chin,living,individual,zyp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zyphe,language,Eurasia,22.52400,93.26400,zyph1238
7867,Zaza,living,macrolanguage,zza,"Indo-European, Indo-Iranian, Iranian, Western,...","Indo-European, Indo-Iranian, Iranian, Western,...","Provinces of Sivas, Tunceli, Bingöl, Erzurum, ...",Unknown,Unknown,https://en.wikipedia.org/wiki/Zaza_language,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zaza1246


In [49]:
merge_v4['glottocode'].isna().sum()

153

In [50]:
merge_v4

,name_x,type,scope,iso6393,family,dialects,regions,off_lang,rec_min_lang,url,Degree of endangerment,speakers,name_y,level,macroarea,latitude,longitude,glottocode
0,Ghotuo,living,individual,aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ghotuo,language,Africa,7.11551,5.95663,ghot1243
1,Alumu-Tesu,living,individual,aab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arum,language,Africa,9.01513,8.55956,alum1246
2,Ari,living,individual,aac,NaN,NaN,NaN,NaN,NaN,NaN,Severely endangered,80.0,Ari,language,Papunesia,-7.95526,142.40000,arii1243
3,Amal,living,individual,aad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Amal,language,Papunesia,-4.04517,141.99500,amal1242
4,Arbëreshë Albanian,living,individual,aae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arbëreshë Albanian,language,Eurasia,38.91100,16.71640,arbe1236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7864,Youjiang Zhuang,living,individual,zyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Youjiang Zhuang,language,Eurasia,23.75000,106.90000,youj1238
7865,Yongnan Zhuang,living,individual,zyn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yongnan Zhuang,language,Eurasia,22.80000,108.00000,yong1275
7866,Zyphe Chin,living,individual,zyp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zyphe,language,Eurasia,22.52400,93.26400,zyph1238
7867,Zaza,living,macrolanguage,zza,"Indo-European, Indo-Iranian, Iranian, Western,...","Indo-European, Indo-Iranian, Iranian, Western,...","Provinces of Sivas, Tunceli, Bingöl, Erzurum, ...",Unknown,Unknown,https://en.wikipedia.org/wiki/Zaza_language,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zaza1246


In [51]:
# indices = merge_v4[merge_v4['latitude'] == np.NaN]
merge_v4 = merge_v4.dropna(subset = ['latitude'])

In [52]:
def remove(s):
    digits = "0123456789"
    for i in digits:
        s = s.replace(i, "")
    return s

In [53]:
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('nan', 'Uknown'))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('[', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(']', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace("'", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace("?", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace("\n", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace("\\n", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(",,", ','))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(" , ", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(" (),", ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: remove(x))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace('nan', 'Uknown'))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace('[', ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace(']', ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace("'", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace("?", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace("\n", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace("\\n", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace(",,", ','))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace(" , ", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: str(x).replace(" (),", ''))
merge_v4['dialects'] = merge_v4['dialects'].apply(lambda x: remove(x))


# return to iso3codes column
merge_v4['speakers'] = merge_v4['speakers'].apply(lambda x: str(x).replace('nan', 'Uknown')) 
merge_v4['speakers'] = merge_v4['speakers'].apply(lambda x: str(x).split('(')[0]) 
merge_v4['off_lang'] = merge_v4['off_lang'].apply(lambda x: str(x).replace('nan', 'Unknown'))
merge_v4['off_lang'] = merge_v4['off_lang'].apply(lambda x: str(x).replace('\xa0', ' '))
merge_v4['off_lang'] = merge_v4['off_lang'].apply(lambda x: str(x).replace('[', ''))
merge_v4['off_lang'] = merge_v4['off_lang'].apply(lambda x: str(x).replace(']', ''))
merge_v4['off_lang'] = merge_v4['off_lang'].apply(lambda x: remove(x))
merge_v4['rec_min_lang'] = merge_v4['rec_min_lang'].apply(lambda x: str(x).replace('nan', 'Unknown'))
merge_v4['rec_min_lang'] = merge_v4['rec_min_lang'].apply(lambda x: str(x).replace('\xa0', ' '))
merge_v4['rec_min_lang'] = merge_v4['rec_min_lang'].apply(lambda x: str(x).replace('[', ''))
merge_v4['rec_min_lang'] = merge_v4['rec_min_lang'].apply(lambda x: str(x).replace(']', ''))
merge_v4['rec_min_lang'] = merge_v4['rec_min_lang'].apply(lambda x: remove(x))

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/240243218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('nan', 'Uknown'))
/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/240243218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('[', ''))
/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/240243218.py:3: SettingWit

In [54]:
for column in merge_v4.columns:
    merge_v4[column] = merge_v4[column].apply(lambda x: x if x is not np.NaN else 'Data unavailable')

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/164051945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4[column] = merge_v4[column].apply(lambda x: x if x is not np.NaN else 'Data unavailable')


In [55]:
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('\n', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('[', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(']', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('\'', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('?', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('nan', 'Unknown'))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('(', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(')', ''))
merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace(',', ', '))

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/2806868617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('\n', ''))
/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/2806868617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4['family'] = merge_v4['family'].apply(lambda x: str(x).replace('[', ''))
/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/2806868617.py:3: SettingWithCop

In [56]:
def create_hierarchy(txt):
    txt = str(txt).split(', ')
    newtxt = ''

    for i in range(len(txt)): 
        newtxt = newtxt + txt[i] + '\n' 

    outtxt = ''
    newtxt = newtxt.split('\n')[:-1]
    for i in range(len(newtxt)):
        outtxt = outtxt + f'<ul><li>{newtxt[i]}</li>'
    num_indents = len(newtxt)
    outtxt = outtxt + '</ul>'*len(newtxt)
    
    return outtxt

merge_v4['family'] = merge_v4['family'].apply(create_hierarchy)
# merge_v4.to_csv('../docs/merged_v3.csv')

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/1985030266.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4['family'] = merge_v4['family'].apply(create_hierarchy)


In [57]:
for column in merge_v4.columns:
    merge_v4[column] = merge_v4[column].apply(lambda x: x if x is not np.NaN else 'Data unavailable')

merge_v4.to_csv('../docs/master_merge.csv')

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_4777/3823496755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_v4[column] = merge_v4[column].apply(lambda x: x if x is not np.NaN else 'Data unavailable')
